In [11]:
import requests
from dotenv import load_dotenv
import os
import json
load_dotenv()

True

# 1. Exchange token and access GCP resources through API

In [2]:
TENANT_ID = os.getenv("TENANT_ID")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
PROJECT_NUMBER = os.getenv("PROJECT_NUMBER")
PROJECT_ID = os.getenv("PROJECT_ID")

In [3]:
os.getenv("PROJECT_NUMBER")

'856284218728'

In [4]:
azure_token_issuer_url = "https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token".format(tenant_id=TENANT_ID)

# 2. Exchange token and access GCP resources through Python libraries

#### Get access_token from Azure Entra ID first. (Note: please register app via Azure and create client_secret if Azure Metadata Server is not accessiblee)

In [5]:
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

In [6]:
data = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'grant_type': 'client_credentials',
    'scope': '{client_id}'.format(client_id=CLIENT_ID) + "/.default"
}

In [7]:
response = requests.post(azure_token_issuer_url, data=data, headers=headers)

In [8]:
azure_access_token = response.json()['access_token']

#### Exchange Azure token for a GCP token with STS - Security Token Service

In [17]:
gcp_sts_url = "https://sts.googleapis.com/v1/token"
headers = {
    'Content-Type': 'application/json'
}
data = {
    "audience": "//iam.googleapis.com/projects/{project_number}/locations/global/workloadIdentityPools/{pool_id}/providers/{provider_id}".format(project_number=PROJECT_NUMBER, pool_id="azure-pool", provider_id="azure"),
    "grantType": "urn:ietf:params:oauth:grant-type:token-exchange",
    "requestedTokenType": "urn:ietf:params:oauth:token-type:access_token",
    "scope": "https://www.googleapis.com/auth/cloud-platform",
    "subjectTokenType": "urn:ietf:params:oauth:token-type:jwt",
    "subjectToken": azure_access_token
}

In [18]:
response = requests.post(gcp_sts_url, data=json.dumps(data), headers=headers)

In [19]:
response.json()

{'access_token': 'ya29.d.c0ASRK0GYM_Cas4DFK4yKFifL1FZFgcd7Q45nD0Ai2lT5SmljIwyj2zpMIWrCpc_xZYqPFvf3sXkrCjL_LA8GziKLyZiUswXciT_hfablahMbkcrixypMakSc90NwzqME78X3ejmU6hx5HhnikTPOHSGu-dzluWpdJp1s11oGF2Y-_OheSwuT3KDgoRlCm5YkOWa4_ryzjGAkU3hIannXWgNlmZkmFvXpcT3n9g7dDeWKUxef-_XioAqh9gyKBesFiNNZadeXlbhS8H9zmVPW-2T5BQ5YcEQTbQF_oM-PhZ-V4k7vNEF9nRd_vYmyycLW07O00otLkuCaylhcVgpJ7cHd7wtVMpuKXJw5WlmZhMBJlwwsduUN0Gl4w78Lu0Ey2X0Luj9ApSnkDVtYDTcxbKSvgZxzG30ZggudcZyOUV9gQTv95tVQJBrRRrCwkrlBUBy6yHqveem0Cw_CjSC4JTJbeXOh2BMK80bcezf0SYRXI3xXsOBYqd4aay_S34lU6pwdZV00cR1lkYj9pU33b5N05b3Fx3ez9DJ1WDenICZbVHaw9YrVDWE1AmF50klRLn3PeMMT66OI4Qm_zfKlrG7zaOh2ClVpAqSwX1Q0_wN-bvkDx2dchK2SL9s3PsKBgmQJ1TT_ScjSxd6FXippZRJ5zL7csyweUCljLKd89Hmblg3eiSgomNePOpsmQJldY-6v2AV77v7aTaWvBNpL_YLPvD4YYbbwNSiUEOjETU11z3mC_Y04VVuF9rEZh0Th06eFzY6YWhER6xUwsC4-VKML-RhqMnzC5PeNlB9LxxvH6fnS27Uky4lOMtneQbFJxpgzKtWTr1V3avzHfHocDUcjVHzPga2bZ-6tZYJwwul3W4jw-FmtGGxCW8mhlSLNE90tWit5FxMhKToBzfG2dl2Pcz5z2jTLcnhn1cQRC5fwnx5MuU6DcgWNScgWIPJMXC2Fgjda7A4TBc